In [1]:
import os
import time 
import pickle

import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

In [2]:
# Root of Project
ROOT = '/home/dev/ws_percept/src'

# Cameras
IMAGE_SIZE  = 240
CAMERAS = ['front', 'back']

# Scene
# [x_min, y_min, z_min, x_max, y_max, z_max] - the metric volume to be voxelized
SCENE_BOUNDS    = (-1.5, -1.5, -1.5, 1.50, 1.5, 1.5)

# Agents
AGENTS = ['Franka0', 'Franka1']

In [3]:
# load data
DATASET_PATH = '../outputs/dualarms_2cam/dualarms_2cam_with_fk.pkl'
with open(DATASET_PATH, 'rb') as f:
    _obs = pickle.load(f)

## Setup PCD, Mesh

In [4]:
def constrain_scene_bounds(data):
    global SCENE_BOUNDS

    nX, nY, nZ, X, Y, Z = SCENE_BOUNDS

    X_mask = (data[:,0] > nX) & (data[:,0]<X)
    Y_mask = (data[:,1] > nY) & (data[:,1]<Y)
    Z_mask = (data[:,2] > nZ) & (data[:,2]<Z)
    mask = (X_mask)&(Y_mask)&(Z_mask) 

    return data[mask]

def get_pcd_and_rgb(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    rgb = camera_obs['rgb'].reshape(-2,3)
    return pcd, rgb

def get_pcd(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    return pcd


obs = _obs[-1]
camera_obs = obs[CAMERAS[0]]
pcd, rgb = get_pcd_and_rgb(camera_obs)

In [5]:
from scipy.spatial.transform import Rotation as R
import cupoch as cph

ROBOT_URDF_PATH = 'outputs/testdata/franka_panda_cupoch_2/panda.urdf'
URDF_JOINT_PREFIX = 'panda_joint'

path = os.path.join(ROOT, ROBOT_URDF_PATH)
kin = cph.kinematics.KinematicChain(path)


def get_tf_matrix(
    translation:np.array,
    rotation:np.array, 
) -> np.array:

    r = R.from_rotvec(rotation)
    r = r.as_matrix()

    tf = np.eye(4)
    tf[:3,:3] = r
    tf[:3,3] = translation

    return tf

def get_mesh_using_forward_kinematics(
    joints_positions:np.array,
    global_position:np.array,
    global_rotation:np.array
):
    global URDF_JOINT_PREFIX

    # create joint_map, used for fk on urdf model
    joint_map = {'%s%d' % (URDF_JOINT_PREFIX, i+1): val for i, val \
                in enumerate(joints_positions)} 

    # manual offset, to be removed 
    offset = np.array([0.0, 0.0, -0.06999997])

    # create transformation matrix
    tf_matrix = get_tf_matrix(global_position+offset, global_rotation)
    
    # do forward kinematics
    poses = kin.forward_kinematics(joint_map, tf_matrix)
    
    # store mesh geometries
    meshes = kin.get_transformed_visual_geometry_map(poses)

    return list(meshes.values())

# agent_obs = obs[AGENTS[0]]
# meshes = get_mesh_using_forward_kinematics(
#     agent_obs['joint_pos'],
#     agent_obs['global_pos'],
#     agent_obs['global_ang'],
# )

## PCR

In [6]:
def get_pcds_for_registration(obs):
    pcds = list()
    camera_obs = obs[CAMERAS[0]]
    pcd, rgb = get_pcd_and_rgb(camera_obs)
    pcd = constrain_scene_bounds(pcd)
    pcds.append(pcd)

    camera_obs = obs[CAMERAS[1]]
    pcd, rgb = get_pcd_and_rgb(camera_obs)
    pcd = constrain_scene_bounds(pcd)
    pcds.append(pcd)
    
    return pcds

pcds = get_pcds_for_registration(obs)

In [7]:


def do_point_cloud_registration(pcds):
    
    # load source and target pointcloud
    source_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[0])
    )
    target_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[1])
    )

    threshold = 0.02 # what does this do?
    trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

    # register pointclouds
    start = time.time()
    reg_p2p = cph.registration.registration_icp(
        source_gpu,
        target_gpu,
        threshold,
        trans_init.astype(np.float32),
        cph.registration.TransformationEstimationPointToPlane(),
    )

    source_gpu.transform(reg_p2p.transformation)


    # remove outliers
    NEIGHBOURS = 2
    source_gpu, ind = source_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)
    target_gpu, ind = target_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)


    elapsed_time = time.time() - start
    print("ICP (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

    # cph.visualization.draw_geometries([source_gpu+target_gpu])

    return source_gpu+target_gpu


pcd = do_point_cloud_registration(pcds)

ICP (GPU) [sec]: 0.07349896430969238


## RBS

In [8]:
def get_bb_from_mesh(
    mesh:cph.geometry.TriangleMesh
) -> cph.geometry.AxisAlignedBoundingBox:

    AABB = mesh.get_axis_aligned_bounding_box()
    if AABB.is_empty():
        print('empty mesh detected')
        return None       
    else:
        return AABB
    

def perform_rbs_on_pointcloud_using_bb(obs, pcd):

    n = len(pcd.points)
    masks = list()
    for agent in AGENTS:
        print('\nAgent: %s' % agent)
        agent_obs = obs[agent]
        
        # get all meshes of the agent using fk
        meshes = get_mesh_using_forward_kinematics(
            agent_obs['joint_pos'],
            agent_obs['global_pos'],
            agent_obs['global_ang'],
        )

        # find points within BBs and create masks
        start = time.time()
        for mesh in meshes:
            bb = get_bb_from_mesh(mesh)

            if bb is not None:
                mask = np.zeros(n, dtype=bool)

                indices_points_within_bb = np.asarray(
                    bb.get_point_indices_within_bounding_box(
                        pcd.points
                    ).cpu()
                )
                if len(indices_points_within_bb)>0:
                    mask[indices_points_within_bb] = True
                    masks.append(mask)

        elapsed_time = time.time() - start
        print("RBS (CPU+GPU) [sec]:", elapsed_time)


    mask = np.column_stack(tuple(masks)).any(axis=1)
    mask = ~mask # inverting mask to get pcd which were not within any BBs
    
    pcd_arr = np.asarray(pcd.points.cpu())
    return  cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcd_arr[mask])
    )

rbs_pcd = perform_rbs_on_pointcloud_using_bb(obs, pcd)


Agent: Franka0
empty mesh detected
RBS (CPU+GPU) [sec]: 0.04869890213012695

Agent: Franka1
empty mesh detected
RBS (CPU+GPU) [sec]: 0.03590059280395508


## Voxelization

In [9]:
def convert_pointcloud_to_voxels(pcd):
    cubic_size = 2.0
    voxel_resolution = 100.0

    # create voxel grid
    start = time.time()
    voxels = cph.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
        pcd,
        voxel_size=cubic_size / voxel_resolution,
        min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
        max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2),
    )
    elapsed_time = time.time() - start
    print("Voxelization (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

    # cph.visualization.draw_geometries([voxels])

    return voxels

voxels = convert_pointcloud_to_voxels(rbs_pcd)

Voxelization (GPU) [sec]: 0.007925748825073242


## Spherization

In [10]:
voxel_points = list(voxels.voxels.cpu().values())

In [11]:
points_as_spheres = list()
for voxel_point in voxel_points:
    points_as_spheres.append(cph.collision.Sphere(1.0, voxel_point.grid_index))

In [12]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
dir(o3d.geometry)

['All',
 'Average',
 'AxisAlignedBoundingBox',
 'Color',
 'DeformAsRigidAsPossibleEnergy',
 'FilterScope',
 'Gaussian3',
 'Gaussian5',
 'Gaussian7',
 'Geometry',
 'Geometry2D',
 'Geometry3D',
 'HalfEdge',
 'HalfEdgeTriangleMesh',
 'Image',
 'ImageFilterType',
 'KDTreeFlann',
 'KDTreeSearchParam',
 'KDTreeSearchParamHybrid',
 'KDTreeSearchParamKNN',
 'KDTreeSearchParamRadius',
 'LineSet',
 'MeshBase',
 'Normal',
 'Octree',
 'OctreeColorLeafNode',
 'OctreeInternalNode',
 'OctreeInternalPointNode',
 'OctreeLeafNode',
 'OctreeNode',
 'OctreeNodeInfo',
 'OctreePointColorLeafNode',
 'OrientedBoundingBox',
 'PointCloud',
 'Quadric',
 'RGBDImage',
 'SimplificationContraction',
 'Smoothed',
 'Sobel3dx',
 'Sobel3dy',
 'Spokes',
 'TetraMesh',
 'TriangleMesh',
 'Vertex',
 'Voxel',
 'VoxelGrid',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'get_rotation_matrix_from_axis_angle',
 'get_rotation_matrix_from_quaternion',
 'get_rotation_matrix_from_xyz',
 'get_rotation_matrix_fr

## Misc

In [14]:
# def get_bb_array_from_mesh(
#     mesh:cph.geometry.TriangleMesh
# ) -> np.array:

#     AABB = mesh.get_axis_aligned_bounding_box()
#     if AABB.is_empty():
#         print('empty mesh detected')
#         return None       
#     else:
#         return np.array(AABB.get_box_points()).transpose(1,0)

# # def get_bb_array_from_mesh(mesh)->np.array:
# #     try:
# #         mesh_points = np.array(mesh.vertices.cpu())
# #         bb = OBB.create_from_points(
# #             o3d.utility.Vector3dVector(mesh_points)
# #         ).get_box_points()
# #         return np.asarray(bb)#.transpose(1,0)
# #     except:
# #         return None

# def get_bb_trace_from_mesh(mesh, color='blue'):
#     bb = get_bb_array_from_mesh(mesh)
#     if bb is not None:
#         return go.Mesh3d(
#             x=bb[0,:],
#             y=bb[1,:], 
#             z=bb[2,:],
#             alphahull=0,
#             opacity=0.5,
#             color=color
#         )
#     else:
#         return None

# def visualize_pointcloud(obs, pcd):
#     pcd = np.asarray(pcd.points.cpu()).transpose(1,0)

#     fig = go.Figure()
#     fig.add_trace(
#         go.Scatter3d(
#             x = pcd[0,:],
#             y = pcd[1,:],
#             z = pcd[2,:],
#             mode='markers',
#             marker=dict(
#                 size=2,
#                 color=pcd[2,:],
#                 colorscale='Turbo',
#             ), 
#         )
#     )
#     agent = AGENTS[1]
#     colors = ['cyan', 'yellow']

#     for agent, color in zip(AGENTS, colors):
#         agent_obs = obs[agent]
#         meshes = get_mesh_using_forward_kinematics(
#             deepcopy(deepcopy(agent_obs['joint_pos'])),
#             deepcopy(deepcopy(agent_obs['global_pos'])),
#             deepcopy(deepcopy(agent_obs['global_ang'])),
#         )

#         for mesh in meshes:
#             try:
#                 trace = get_bb_trace_from_mesh(mesh, color=color)
#                 if trace is not None:
#                     fig.add_trace(trace)
#             except:
#                 pass

#     return fig


# visualize_pointcloud(obs, pcd)